In [ ]:
pip install yfinance
pip install kafka-python

In [1]:
import pandas as pd
import yfinance as yf
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json

In [3]:
tickers = ['MSFT', 'AAPL', 'GOOG', 'AMZN']  # Add more tickers as needed
all_data = []  # Create an empty list to store data for all tickers

start = '2020-1-1'
end = '2024-11-23'

for ticker in tickers:
    dat = yf.Ticker(ticker)
    dat_price_df = dat.history(interval='1d', start=start, end=end)
    dat_price_df = dat_price_df.reset_index()
    dat_price_df['stock_name'] = ticker  # Assign the current ticker
    dat_price_df['Date'] = pd.to_datetime(dat_price_df['Date'])  # Convert to timestamp
    sel_col = ['stock_name','Date','Open','High','Low','Close','Volume']
    dat_price_df = dat_price_df[sel_col]
    all_data.append(dat_price_df)  # Append data for current ticker

# Concatenate all data into a single DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Convert `Date` to string format (e.g., ISO 8601) for Kafka compatibility
final_df['Date'] = final_df['Date'].dt.strftime('%Y-%m-%dT%H:%M:%S')


In [4]:
producer = KafkaProducer(bootstrap_servers=['18.136.104.39:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [ ]:
while True:
    dict_stock = final_df.sample(1).to_dict(orient="records")[0]
    producer.send('stock', value=dict_stock)
    sleep(1)

In [6]:
producer.flush() #clear data from kafka server